In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
"""
Running MNIST pipeline entirely online, rather than from csv files like currently done in the experiments folder
"""

# from gcp_hpo.smart_search import SmartSearch
# from ..smart_search import SmartSearch # Alec edit
import numpy as np
import math
import pandas as pd

In [ ]:
np.set_printoptions(edgeitems=100)

In [ ]:
# Set parameters
blur_ksize = 1
blur_sigma = 4
pca_dim = 60
degree_poly = 3
gamma = 10**(-3)

In [ ]:
def import_data():
    """
    Steps:
        1. Import MNIST data (from Kaggle to be consistent with Sebastien)
        2. Blur the training images
        3. Reduce image matrices by principal component analysis
        4. classify using polynomial SVM
    """

    # Import MNIST data
    data = np.loadtxt('/Users/aandersonlaptop/Desktop/MNIST_train.csv',skiprows=1,delimiter=',') # way slower than pandas
    new_x = data[:,1:]
    new_y = data[:,0]
#     print 'original',data
    print 'Y',new_y
    print 'X',new_x
    return new_x,new_y
    
    
X_train,y_train = import_data()

In [ ]:
from scipy.ndimage.filters import gaussian_filter 

def gaussian_blur(X,stddev,k_size):
    # Old way
#     output = np.empty(X.shape)
#     output[:] = np.NAN
# #     print 'original',output
# #     for i in range(X.shape[0]):
# #         a = np.array([X[i][0]])
# #         b = gaussian_filter(X[i][1:],sigma=stddev,order=0,truncate=k_size)
# #         output[i,] = np.concatenate((a,b),axis=0)
        
# #     print 'output',output
# #     return output

    # New way
    output = gaussian_filter(X,sigma=stddev,order=0,truncate=k_size)
#     print 'output',output
    return output

blurred_X_train = gaussian_blur(X_train,stddev=blur_sigma,k_size=blur_ksize)   

In [ ]:
from sklearn.decomposition import PCA

def do_pca(X,num_components):
    pca = PCA(n_components = num_components)
#     pca.fit(X.T)
#     print pca.components_.T.shape
#     return pca.components_.T
    return pca.fit_transform(X)
    
    
pca_X_train = do_pca(blurred_X_train,num_components=pca_dim)

In [ ]:
### Random sampling ###
def random_sample(X,y,num_samples):
    idx = np.random.randint(X.shape[0],size=num_samples)
#     print 'idx',idx
    return X[idx],y[idx]

X_sample,y_sample = random_sample(pca_X_train,y_train,15000)
print 'shape',X_sample.shape
print 'shape',y_sample.shape

In [ ]:
from sklearn.svm import SVC

def do_svm(X,y,degree,gamma_coeff):
    # Old way
#     clf = SVC()
# #     print len(data_3[:,0])
#     data_3[:,0] = data_3[:,0].astype(int)
# #     print np.unique(data_3[:,0])
#     clf.fit(data_3[:,1:],data_3[:,0])
#     return clf.score(data_3[:,1:],data_3[:,0]) # TODO this is just the training accuracy

    # New way
    clf = SVC(kernel='poly',degree=degree,gamma=gamma_coeff)
    clf.fit(X,y)
    print 'accuracy on training set',clf.score(X,y)
    return clf

model = do_svm(pca_X_train,y_train,degree=degree_poly,gamma_coeff=gamma)
# model = do_svm(X_train[:5000,:],y_train[:5000],degree=degree_poly,gamma_coeff=gamma)
# model = do_svm(X_sample,y_sample,degree=degree_poly,gamma_coeff=gamma)

In [ ]:
%time print 'go'

In [ ]:
############################# Test model ############################
def import_test_data():
    # Import MNIST data
    data = np.loadtxt('/Users/aandersonlaptop/Desktop/MNIST_test.csv',skiprows=1,delimiter=',') # way slower than pandas
    new_x = data[:,1:]
    new_y = data[:,0]
    print 'Y',new_y
    print 'X',new_x
    return new_x,new_y

X_test,y_test = import_data()

In [ ]:
### Test model ###

blurred_X_test = gaussian_blur(X_test,stddev=blur_sigma,k_size=blur_ksize)
pca_X_test = do_pca(blurred_X_test,num_components=pca_dim)
print model.score(pca_X_test,y_test)
# print model.score(X_test,y_test)